In [80]:
import numpy.random as sim
import numpy as np
def arrondi(x):
    return int(x)+ (int(x*10)%10)/10
data = [arrondi(-10+2*sim.randn()) for i in range(100000)]
data

[-9.3,
 -6.8,
 -9.7,
 -9.4,
 -5.6,
 -13.0,
 -9.2,
 -13.4,
 -7.6,
 -6.4,
 -8.0,
 -7.1,
 -10.3,
 -14.0,
 -11.4,
 -9.0,
 -8.2,
 -10.8,
 -12.9,
 -9.2,
 -11.0,
 -8.1,
 -9.5,
 -8.0,
 -3.2,
 -8.4,
 -8.5,
 -10.0,
 -7.3,
 -6.8,
 -10.1,
 -8.8,
 -10.3,
 -7.4,
 -11.7,
 -9.3,
 -10.2,
 -7.8,
 -9.3,
 -9.4,
 -7.0,
 -9.3,
 -6.7,
 -9.7,
 -9.7,
 -10.6,
 -8.8,
 -10.0,
 -11.7,
 -10.2,
 -9.7,
 -8.2,
 -11.4,
 -11.8,
 -10.4,
 -6.2,
 -8.8,
 -7.6,
 -10.9,
 -5.7,
 -7.7,
 -7.8,
 -11.2,
 -9.7,
 -5.9,
 -7.4,
 -10.2,
 -8.2,
 -9.0,
 -9.8,
 -6.8,
 -9.7,
 -8.1,
 -8.4,
 -9.1,
 -11.1,
 -5.1,
 -6.6,
 -6.3,
 -12.3,
 -9.2,
 -7.9,
 -7.2,
 -6.9,
 -9.5,
 -10.6,
 -10.9,
 -7.5,
 -9.0,
 -8.1,
 -6.1,
 -9.4,
 -10.3,
 -11.1,
 -9.0,
 -7.8,
 -9.1,
 -9.8,
 -10.5,
 -8.3,
 -7.8,
 -9.4,
 -9.3,
 -13.7,
 -7.9,
 -9.5,
 -5.1,
 -7.0,
 -8.8,
 -9.2,
 -10.2,
 -8.3,
 -7.9,
 -6.5,
 -11.9,
 -11.9,
 -11.0,
 -8.2,
 -9.7,
 -11.2,
 -6.5,
 -7.7,
 -10.5,
 -6.2,
 -7.9,
 -9.1,
 -8.0,
 -7.6,
 -7.8,
 -6.9,
 -9.7,
 -12.0,
 -7.8,
 -9.8,
 -9.2,
 -10.9,
 -12.2,
 

In [81]:
def get_freq(data):
    sorted_data = sorted(data)
    freq = []
    x = []
    n = len(sorted_data)
    for i in range(len(sorted_data)):
        if i == 0:
            freq.append(1/n)
            x.append(sorted_data[i])
        elif sorted_data[i-1]!=sorted_data[i]:
            freq.append(freq[-1]+1/n)
            x.append(sorted_data[i])
        else:
            freq[-1] += 1/n
    return freq, x

In [ ]:
def VaR2(data):
    sorted_data = sorted(data)
    freq = []
    x = []
    n = len(sorted_data)
    for i in range(len(sorted_data)):
        if i == 0:
            freq.append(1/n)
            x.append(sorted_data[i])
            if (freq[-1] >= 1-alpha) :
                return -sorted_data[i]
        elif sorted_data[i-1]!=sorted_data[i]:
            freq.append(freq[-1]+1/n)
            x.append(sorted_data[i])
            if (freq[-1] >= 1-alpha) :
                return -sorted_data[i]
        else:
            freq[-1] += 1/n
            if (freq[-1] >= 1-alpha) :
                return -sorted_data[i]
    return freq, x

In [90]:
def VaR(data, alpha):
    freq, x = get_freq(data)
    for i in range(len(freq)):
        if (freq[i] >= 1-alpha) :
            return -x[i]

In [95]:
VaR(data, 0.8)

10.7

In [96]:
def tVaR(data, alpha, precision):
    alphas = np.linspace(alpha,1-(1-alpha)/precision, precision)
    sum_VaR = 0
    for alpha_i in alphas:
        sum_VaR += VaR(data, alpha_i)
    
    sum_VaR = sum_VaR / precision

    return sum_VaR

In [98]:
tVaR(data,0.8,100)

11.849